In [2]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sktime.datasets import load_basic_motions

from sktime.classification.interval_based import DrCIF

from sklearn.metrics import classification_report, precision_recall_fscore_support 
from sklearn.model_selection import GridSearchCV, KFold

import warnings
warnings.filterwarnings('ignore')

X_train, y_train = load_basic_motions(split='train', return_type='numpy3D')
X_test, y_test = load_basic_motions(split='test', return_type='numpy3D')

## Diverse Representation Canonical Interval Forest (DrCIF)

In [27]:
drcif = DrCIF(
    n_estimators=5,
    n_intervals=2,        # Number of intervals per representation, per tree
    att_subsample_size=8, # Number of Catch22 features
    min_interval=3,
    max_interval=None,    # Default is half of series length
    base_estimator="CIT",
    n_jobs=-1,
    random_state=42
)

params = {
    "n_estimators": [1, 2, 3, 4, 5]
}

tuned_drcif = GridSearchCV(
    drcif,
    params,
    cv=KFold(n_splits=10)
)

tuned_drcif.fit(X_train, y_train)
y_pred = tuned_drcif.predict(X_test)

clf_report = classification_report(y_test, y_pred)

precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')

print(tuned_drcif.best_params_)
print(clf_report)

{'n_estimators': 3}
              precision    recall  f1-score   support

   badminton       1.00      1.00      1.00        10
     running       1.00      1.00      1.00        10
    standing       0.91      1.00      0.95        10
     walking       1.00      0.90      0.95        10

    accuracy                           0.97        40
   macro avg       0.98      0.97      0.97        40
weighted avg       0.98      0.97      0.97        40

